In [45]:
import pandas as pd
import numpy as np

In [46]:
path_loanData="Datasets/loan_data.csv"
path_repaymentData="Datasets/repayment_data.csv"

In [47]:
x = pd.read_csv(path_loanData)

In [48]:
x['disbursed_date']

0        24/10/2021
1        10/11/2021
2               NaN
3        20/11/2021
4        26/11/2021
            ...    
12893           NaN
12894           NaN
12895           NaN
12896           NaN
12897           NaN
Name: disbursed_date, Length: 12898, dtype: object

In [49]:
x[x['loan_id']=='loan_8836810'][['application_date', 'disbursed_date', 'paid_date']]

,application_date,disbursed_date,paid_date
3092,12/11/2021 11:34,12/11/2021,15/11/2021


In [50]:
df_loanData = pd.read_csv(path_loanData).astype({
    'customer_id': 'str',
    'merchant_id': 'str',
    'loan_id': 'str',
    'currency': 'str',
    'transaction_fee': 'float64',
    'merchant_status': 'str',
    'store_id': 'str',
    'term': 'float64',
    'risk_score': 'float64',
    'loanamount': 'float64',
    'city': 'category',
    'state': 'category',
    'channel': 'category',

})
df_loanData['application_date'] = pd.to_datetime(df_loanData['application_date'], format='%d/%m/%Y %H:%M')
df_loanData['disbursed_date'] = pd.to_datetime(df_loanData['disbursed_date'], format='%d/%m/%Y')
df_loanData['paid_date'] = pd.to_datetime(df_loanData['paid_date'], format='%d/%m/%Y')
df_loanData['first_vintage_pay'] = pd.to_datetime(df_loanData['first_vintage_pay'], format='%d/%m/%Y')



df_loanData = df_loanData.sort_values('disbursed_date').reset_index(drop=True)


In [51]:
df_loanData['disbursed_date'].min()

Timestamp('2021-10-01 00:00:00')

In [52]:
df_loanData[df_loanData['loan_id']=='loan_8836810'][['application_date', 'disbursed_date', 'paid_date']]

,application_date,disbursed_date,paid_date
673,2021-11-12 11:34:00,2021-11-12,2021-11-15


In [53]:
df_repaymentData = pd.read_csv(path_repaymentData)

df_repaymentData['installment_duedate'] = pd.to_datetime(df_repaymentData['installment_duedate'], format='%d/%m/%Y')
df_repaymentData['repaid_date'] = pd.to_datetime(df_repaymentData['repaid_date'], format='%d/%m/%Y')

In [54]:
pd.read_csv(path_repaymentData)['installment_duedate']

0        31/03/2022
1        16/05/2022
2        02/05/2022
3        31/05/2022
4        18/04/2022
            ...    
29566    15/07/2022
29567    16/05/2022
29568    30/06/2022
29569    15/06/2022
29570    02/05/2022
Name: installment_duedate, Length: 29571, dtype: object

# Buckets risk_score & fraud_score

# Recurrence 1

In [55]:
#df_loanData[df_loanData['is_disbursed']==True]

df_loanData.sort_values('disbursed_date', inplace=True)

In [56]:
df_loanData.reset_index(drop=True, inplace=True)

In [57]:
df_loanData['dln'] = df_loanData.apply(lambda row: 1 if row['is_disbursed']==True else 0, axis=1)
df_loanData['recurrence_1'] = 0

In [58]:
df_loanData.head()
df_loanData[(df_loanData['customer_id']=='6157ygy67h8gzh650775091g')]

,customer_id,merchant_id,loan_id,application_date,currency,transaction_fee,merchant_status,store_id,term,risk_score,...,age,gender,tel_number,email,merchant_name,merchant_size,store_name,fraud_score,dln,recurrence_1
4,6157ygy67h8gzh650775091g,726183834821532,loan_4448912,2021-10-01 22:07:00,MXN,0.043,7 Live,nan,4.0,0.7,...,23,H,3336461467,email_1589@gmail.com,merchant_name_82,small,NaN,0.9,1,0
492,6157ygy67h8gzh650775091g,652281012039432,loan_6458386,2021-11-09 09:30:00,MXN,0.050,7 Live,nan,2.0,0.9,...,23,H,3336461467,email_1589@gmail.com,merchant_name_119,large,NaN,0.9,1,0
8626,6157ygy67h8gzh650775091g,726183834821532,loan_5673040,2022-01-21 14:11:00,MXN,NaN,7 Live,nan,NaN,NaN,...,23,H,3336461467,email_1589@gmail.com,merchant_name_82,small,NaN,0.9,0,0
10830,6157ygy67h8gzh650775091g,726183834821532,loan_5868496,2022-01-18 13:55:00,MXN,NaN,7 Live,nan,NaN,NaN,...,23,H,3336461467,email_1589@gmail.com,merchant_name_82,small,NaN,0.5,0,0
10981,6157ygy67h8gzh650775091g,652281012039432,loan_7344024,2021-11-08 16:09:00,MXN,NaN,7 Live,nan,NaN,0.9,...,23,H,3336461467,email_1589@gmail.com,merchant_name_119,large,NaN,1.0,0,0


In [59]:
for customer in df_loanData['customer_id'].unique():
    for i,j in enumerate(df_loanData[(df_loanData['customer_id']==customer)]['dln'].tolist()):
        if j!=0:
            index=df_loanData[df_loanData['customer_id']==customer].index
            df_loanData['recurrence_1'][index[i]]=i+j   
        else:
            df_loanData['recurrence_1'][index[i]]=j

<ipython-input-59-c4adc397bbb0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loanData['recurrence_1'][index[i]]=i+j
<ipython-input-59-c4adc397bbb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loanData['recurrence_1'][index[i]]=j


In [60]:
df_loanData[(df_loanData['recurrence_1']==0)][['customer_id', 'disbursed_date', 'is_disbursed', 'dln', 'recurrence_1']]
#df_loanData[['customer_id', 'disbursed_date', 'is_disbursed', 'dln', 'recurrence_1']]

,customer_id,disbursed_date,is_disbursed,dln,recurrence_1
4934,61943hyxi6z2yhy1hz0099x7,NaT,False,0,0
4935,6188gi6x3g3738yi6gzy92h0,NaT,False,0,0
4936,61z333x1x586xgz7058zi253,NaT,False,0,0
4937,618zyyyx94gixz88h4y44050,NaT,False,0,0
4938,619x990z3i8207844xy5g79g,NaT,False,0,0
...,...,...,...,...,...
12893,623364385hg5746064044532,NaT,False,0,0
12894,61g7818651727zx0h0zg2yy2,NaT,False,0,0
12895,618ii8xg161i66z741xx1xhx,NaT,False,0,0
12896,617101iy3228635663724i91,NaT,False,0,0


# Recurrence 2

In [61]:
df_loanData['recurrence_2'] = 0

In [62]:
df_allData = df_loanData.merge(df_repaymentData, how='left', left_on='loan_id', right_on='loan_id')

In [63]:
for loan in df_loanData['loan_id'].unique():
    index = df_loanData[df_loanData['loan_id']==loan].index
    if (df_loanData[df_loanData['loan_id']==loan]['recurrence_1'].iloc[0]==1) or (df_loanData[df_loanData['loan_id']==loan]['recurrence_1'].iloc[0]==0):
        df_loanData['recurrence_2'][index[i]]=False
    else:
        customer_id = df_allData[df_allData['loan_id']==loan]['customer_id'].unique()[0]
        disbursed_date = df_loanData[df_loanData['loan_id']==loan]['disbursed_date'].iloc[0]
        if (df_allData[(df_allData['customer_id']==customer_id) & (df_allData['paid_date']<disbursed_date)].size!=0) or (df_allData[(df_allData['customer_id']==customer_id) & (df_allData['installment_number']>=3)].size!=0):
            df_loanData['recurrence_2'][index[i]]=True
        else:
            df_loanData['recurrence_2'][index[i]]=False

<ipython-input-63-7652e76259aa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loanData['recurrence_2'][index[i]]=False
c:\Users\AngelGG\miniconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-63-7652e76259aa>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loanData['recurrence_2'][index[i]]=True
<ipython-input-63-7

In [64]:
#df_loanData['recurrence_2'].unique()
df_loanData[df_loanData['recurrence_2']==False]

,customer_id,merchant_id,loan_id,application_date,currency,transaction_fee,merchant_status,store_id,term,risk_score,...,gender,tel_number,email,merchant_name,merchant_size,store_name,fraud_score,dln,recurrence_1,recurrence_2
0,61579726682i3106gxx16x5g,522186628800710,loan_8444478,2021-10-01 18:31:00,MXN,0.040,7 Live,nan,12.0,0.2,...,H,3351318488,email_2607@gmail.com,merchant_name_2,medium,NaN,0.3,1,1,False
1,614020131yh5g4xyi73154ig,522186628800710,loan_0617008,2021-10-01 22:46:00,MXN,0.040,7 Live,nan,12.0,0.8,...,H,3313691637,email_567@gmail.com,merchant_name_2,medium,NaN,0.9,1,1,False
2,6155x1412z1x1g082834ghz9,716945251115144,loan_3959620,2021-10-01 12:02:00,MXN,0.040,7 Live,nan,8.0,0.8,...,M,3333753900,email_821@gmail.com,merchant_name_10,small,NaN,0.5,1,1,False
3,5x2088xh13yh2195ygixx279,522186628800710,loan_4292068,2021-10-01 15:18:00,MXN,0.040,7 Live,nan,12.0,0.6,...,M,3315874580,email_1424@gmail.com,merchant_name_2,medium,NaN,0.5,1,1,False
4,6157ygy67h8gzh650775091g,726183834821532,loan_4448912,2021-10-01 22:07:00,MXN,0.043,7 Live,nan,4.0,0.7,...,H,3336461467,email_1589@gmail.com,merchant_name_82,small,NaN,0.9,1,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12893,623364385hg5746064044532,745738015615046,loan_7867984,2022-04-06 17:34:00,MXN,NaN,7 Live,nan,NaN,NaN,...,M,3332912274,email_1645@gmail.com,merchant_name_687,large,NaN,0.3,0,0,False
12894,61g7818651727zx0h0zg2yy2,849409189622446,loan_5715188,2022-04-11 08:28:00,MXN,NaN,7 Live,nan,NaN,NaN,...,H,3380840148,email_842@gmail.com,merchant_name_713,medium,NaN,0.1,0,0,False
12895,618ii8xg161i66z741xx1xhx,682650767730466,loan_7605196,2022-04-12 13:41:00,MXN,NaN,8.1 Cancelled by Kueski,nan,NaN,NaN,...,H,3311321340,email_27@gmail.com,merchant_name_48,extra-large,NaN,0.2,0,0,False
12896,617101iy3228635663724i91,642155937014514,loan_9610480,2022-04-16 19:02:00,MXN,NaN,7 Live,nan,NaN,0.9,...,M,3304524527,email_2797@gmail.com,merchant_name_85,medium,NaN,0.8,0,0,False


In [65]:
df_loanData[df_loanData['loan_id']=='loan_4153030']['recurrence_1'].iloc[0]

1

In [66]:
df_allData[(df_allData['customer_id']=='618208xh6x7x160y0z5hhz8h') & (df_allData['paid_date']<df_loanData[df_loanData['loan_id']=='loan_4153030']['disbursed_date'].iloc[0])].size!=0

False

In [67]:
df_allData[(df_allData['customer_id']=='618208xh6x7x160y0z5hhz8h') & (df_allData['installment_number']>=3)].size!=0

False

In [68]:
df_loanData[df_loanData['loan_id']=='loan_8836810'][['disbursed_date', 'paid_date']]

,disbursed_date,paid_date
627,2021-11-12,2021-11-15


In [69]:
df_allData[df_allData['customer_id']=='618y9xx46g7y1x62y8970hxz'][['disbursed_date', 'term', 'paid_date', 'installment_number', 'recurrence_1', 'recurrence_2']]

,disbursed_date,term,paid_date,installment_number,recurrence_1,recurrence_2
3975,2021-11-11,8.0,2022-03-16,5.0,1,0
3976,2021-11-11,8.0,2022-03-16,3.0,1,0
3977,2021-11-11,8.0,2022-03-16,8.0,1,0
3978,2021-11-11,8.0,2022-03-16,2.0,1,0
3979,2021-11-11,8.0,2022-03-16,1.0,1,0
3980,2021-11-11,8.0,2022-03-16,6.0,1,0
3981,2021-11-11,8.0,2022-03-16,4.0,1,0
3982,2021-11-11,8.0,2022-03-16,7.0,1,0
9701,2021-11-30,8.0,2022-03-31,7.0,2,0
9702,2021-11-30,8.0,2022-03-31,5.0,2,0


In [70]:
df_allData[(df_allData['customer_id']=='618208xh6x7x160y0z5hhz8h')][['risk_score']]

,risk_score
2351,0.7
2352,0.7


In [71]:
df_allData[(df_allData['customer_id']=='61576054h2x2h8zi1817i690') & (df_allData['paid_date']<df_loanData[df_loanData['loan_id']=='loan_9572502']['disbursed_date'].iloc[0])]

,customer_id,merchant_id,loan_id,application_date,currency,transaction_fee,merchant_status,store_id,term,risk_score,...,dln,recurrence_1,recurrence_2,installment_id,installment_number,installment_duedate,repaid_date,capital_due,capital_paid,interests_paid


In [72]:
df_loanData.groupby(by=['customer_id', 'risk_score'])[['loanamount']].sum().head(20)

loanamount
customer_id              risk_score            
5727h5x517872168842080gz 0.9          1371.7400
5754y0gixi3yx76xyyz9y1x4 0.8          1458.5400
                         0.9         10152.8400
57716216976g4371ih51z4y5 0.5          3650.0000
579ig10837i9ii9157g76139 0.7          2106.4000
57igxh5i387h424x80hyy9z8 0.3          4479.6800
58770683135485x75g3g0i02 0.4          1532.2700
5890z0z67yy3x96z189xzig5 0.6          4215.0200
                         0.8          4215.0200
                         0.9          4379.2700
58iiy7z593y80056x708g857 0.7          5521.8300
58z7x8y866x95z508zg0i274 0.8          3030.8870
58zg4i7326gzxii319h91654 0.0         10219.2700
58zhhz3611291xz755h4xhy8 0.1          3429.5400
                         0.2         11337.6300
                         0.3         20351.6600
5907yg60hyhh6y1g63405h4x 0.8           710.5090
590yzxy9ihxy3841xx54647y 0.8           433.8390
593i0i291hxy16i47y7746g7 0.2          3136.0362
                         0.3         17245.5000

In [73]:
df_loanData['recurrence_2'] = df_loanData['recurrence_2'].astype('bool')

In [74]:
df_loanData.dtypes

customer_id                  object
merchant_id                  object
loan_id                      object
application_date     datetime64[ns]
currency                     object
transaction_fee             float64
merchant_status              object
store_id                     object
term                        float64
risk_score                  float64
disbursed_date       datetime64[ns]
loanamount                  float64
paid_date            datetime64[ns]
city                       category
state                      category
channel                    category
is_disbursed                   bool
first_vintage_pay    datetime64[ns]
full_name                    object
age                           int64
gender                       object
tel_number                    int64
email                        object
merchant_name                object
merchant_size                object
store_name                   object
fraud_score                 float64
dln                         

# Export transform data

In [75]:
df_loanData.to_csv('Datasets/Transformloan_data.csv', index=False)

In [76]:
df_repaymentData.to_csv('Datasets/Transformrepayment_data.csv', index=False)